## Copy segmentation from one scan to another

This notebook demonstrates copying segmentation from one scan to another. Scan and segmentation in the example are read from Nifti, but can also be loaded from DICOM format.

#### Define Paths for scans and segmentation

In [ ]:
origNiiScan = r'/path/to/original_scan.nii'
origNiiStr = r'/path/to/original_seg.nii'
resampNiiScan = r'/path/to/smore_resamp_scan.nii'
outputNiiStr = r'/path/to/smore_resamp_str.nii.gz'

#### Import scans and segmentation to planC

In [ ]:
from cerr import plan_container as pc
planC = pc.load_nii_scan(origNiiScan)
planC = pc.load_nii_scan(resampNiiScan, planC)
assocScanNum = 0 # index of the original scan
structName = 'tumor'
planC = pc.load_nii_structure(origNiiStr, assocScanNum, planC, labels_dict={1: structName})

#### Copy segmentation from the original scan to the resampled scan

In [ ]:
from cerr.dataclasses import structure as cerrStruct
structNum = 0
scanNum = 1
planC = cerrStruct.copyToScan(structNum, scanNum, planC)

#### Visualize scan and segmentation

In [ ]:
from cerr import viewer as vwr
import numpy as np
scanNumList = [1]
doseNumList = []
strNumList = [1]
displayMode = '2d' # 'path' or 'surface'
viewer, scan_layer, dose_layer, struct_layer = \
    vwr.show_scan_struct_dose(scanNumList, strNumList, doseNumList, planC, displayMode)

#### Export segmentation to Nifti

In [ ]:
structNum = len(planC.structure)-1 # assume the structure to export is the last one in the list
planC.structure[structNum].save_nii(outputNiiStr, planC)